 Import Required Libraries

In [1]:
import ee
import geemap
import pandas as pd
from IPython.display import Image

Authenticate Google Earth Engine

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AZEOvhV_OrpIsbxvGh2s7WiCWq3YBb3CCNAPdNEgfYZew5iq4rBXLYmuOKw

Successfully saved authorization token.


Import Data

In [4]:
#Alaska boundary
alaska_bound = ee.FeatureCollection('TIGER/2018/States').filter(ee.Filter.eq('STUSPS','AK'))

#create section of Ykdelta
lon = 62.83
lat = -153
poi = ee.Geometry.Point(lon,lat)
poi_reprojected = poi.transform('EPSG:4269')
box = [-165.3,62.2,-162.5,63.4]
roi =  ee.Geometry.Rectangle(box)

In [5]:
#Import Digital Elevation Model
Dataset = ee.Image('USGS/3DEP/10m')
Elevation = Dataset.select('elevation')

#change alaska boundary to proper coordinate system
alaska_bound_reprojected = alaska_bound.geometry().transform('EPSG:4269')

#clip elevation to just alaska
Elevation_Alaska = Elevation.clip(alaska_bound_reprojected)

url = Elevation_Alaska.getThumbUrl({
    'min': 0, 'max': 2500 ,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})

# Display
Image(url=url)

In [6]:
Elevation_Alaska_reprojected = Elevation_Alaska.reproject(crs='WGS84')

#get download link
link = Elevation_Alaska.getDownloadURL({
    'scale' : 70,
    'crs' : 'EPSG:4269',
    'fileFormat': 'GeoTIFF',
    'region': roi,
    'name': 'elevation'
})
print(link)

url = Elevation_Alaska.getThumbUrl({
    'min': 0, 'max': 2500 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/9c9a4c1bb69f4e545decb8e385333006-072fec0f95b75bd3624c44c4c24da6ba:getPixels


Get Slope from DEM

In [7]:
Slope_Alaska = ee.Terrain.slope(Elevation_Alaska)


url = Slope_Alaska.getThumbUrl({
    'min': 0, 'max': 90 ,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})

# Display
Image(url=url)

In [8]:
#get download link

link = Slope_Alaska.getDownloadURL({
    'scale' : 70,
    'crs' : 'EPSG:4269',
    'fileFormat': 'GeoTIFF',
    'region': roi,
    'name': 'slope'
})
print(link)

url = Slope_Alaska.getThumbUrl({
    'min': 0, 'max': 90 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/46e55a7002689b95176f25dddb543d45-b9259592ee906b597c7a60695f6b1fab:getPixels


In [9]:
Aspect_Alaska = ee.Terrain.aspect(Elevation_Alaska)

url = Aspect_Alaska.getThumbUrl({
    'min': 0, 'max': 360 ,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})

# Display
Image(url=url)

In [10]:
#get download link

link = Aspect_Alaska.getDownloadURL({
    'scale' : 70,
    'crs' : 'EPSG:4269',
    'fileFormat': 'GeoTIFF',
    'region': roi,
    'name': 'aspect'
})
print(link)

url = Aspect_Alaska.getThumbUrl({
    'min': 0, 'max': 360 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/5b97ffd766d4204ac066d1b2652aa056-6a81af676b69cb556bf1982551582d3c:getPixels


In [11]:
#NDVI
Dataset = ee.ImageCollection('MODIS/061/MYD13Q1').filterDate('2018-01-01','2018-02-01')

NDVI = Dataset.select('NDVI')
#NDVI_Alaska = NDVI.clip('alaska_bound_reprojected')
NDVI_Alaska = NDVI.mosaic().clip(alaska_bound)
url = NDVI_Alaska.getThumbUrl({
   'min': -2000, 'max': 10000 ,'region':alaska_bound.geometry(), 'dimensions':2048,
   'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

In [12]:
for i in range(1965,2020,5):
    Start_Date = str(i) + '-01-01'
    End_Date = str(i+4) + '-12-31'

    Dataset = ee.ImageCollection('MODIS/061/MYD13Q1').filterDate(Start_Date,End_Date).select('NDVI')
    NDVI_Alaska = NDVI.mosaic().clip(alaska_bound)
    #get download link
    link = NDVI_Alaska.getDownloadURL({
        'scale' : 50,
        'crs' : 'EPSG:4269',
        'fileFormat': 'GeoTIFF',
        'region': roi,
        'name': 'NDVI_' + str(i)     
    })
    print(Start_Date +": " + link)

url = NDVI_Alaska.getThumbUrl({
    'min': -2000, 'max': 10000 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)
    

1965-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/6ddcb4db470e68d32bed4ba8ae8cc354-4e0ebfec3bc2f17a8e108f0a49b64a46:getPixels
1970-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/f38e8c3c01b35b51a07722f97e9b1c48-5a43ed99f268e4e67b9c52bf1b8519fe:getPixels
1975-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/75198363dc5be303dde0a25e5d3e6273-42aa8d9f1bb38256698c204d8b74fb63:getPixels
1980-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/9f58b9d9555343262881d58df279e942-7eee1103979204fc14edede337300701:getPixels
1985-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/9dc4c018627cd226563b13729e450827-1e9f61afba458861f7157c05ebe9d1b8:getPixels
1990-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/3b538b032947d3a9f88f1d6546db0cfd-428e3caf64dc37aa68fe053

In [87]:
Dataset = ee.ImageCollection('NOAA/CDR/AVHRR/NDVI/V5')
NDVI = Dataset.select('NDVI')
NDVI_test = NDVI.first()
url = NDVI_test.getThumbUrl({
   'min': 0, 'max': 8000, 'dimensions':1024,
   'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)